In [1]:
!pip install -q faker openai

In [2]:
import sys
import random
import string
import re 
import time
import asyncio
from functools import reduce
from pathlib import Path

import torch
import pandas as pd
import numpy as np
import textwrap
from openai import OpenAI, AsyncOpenAI
from IPython.display import Markdown
import matplotlib.pyplot as plt
from kaggle_secrets import UserSecretsClient
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from tqdm.auto import tqdm
from faker import Faker  #generates fake data
from faker.providers import BaseProvider
from jinja2 import Template
from spacy.lang.en import English

2024-03-12 02:27:55.463086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 02:27:55.463234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 02:27:55.604993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
N_GENERATE_ESSAY = 100
MODEL = "gpt-4-turbo-preview"

In [4]:
train_df = pd.read_json("/kaggle/input/pii-detection-removal-from-educational-data/train.json")
reduce(lambda i, j: i | j, [set(row["labels"]) for _, row in train_df.iterrows()])

{'B-EMAIL',
 'B-ID_NUM',
 'B-NAME_STUDENT',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'B-URL_PERSONAL',
 'B-USERNAME',
 'I-ID_NUM',
 'I-NAME_STUDENT',
 'I-PHONE_NUM',
 'I-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'O'}

In [5]:
def extract_real_words(label_name):
    real_words = set()
    for _, row in train_df.iterrows():
        s = ""
        for token, label, space in zip(row["tokens"], row["labels"], row["trailing_whitespace"]):
            if label not in (f"B-{label_name}", f"I-{label_name}"):
                if s:
                    real_words.add(s.strip())
                    s = ""
                continue
            space = " " if space else ""
            if label == f"B-{label_name}":
                if s:
                    real_words.add(s.strip())
                s = token + space
            else:
                s += token + space
        if s:
            real_words.add(s.strip())
    return real_words

#### UserID

In [6]:
class UserIdMixIn:
    num_low: int = int(1e2)
    num_high: int = int(1e12)
    beta_a: float = 1.0
    beta_b: float = 5.0
    
    @property
    def pii_userid(self) -> str:
        try:
            return self._pii_userid
        except AttributeError:
            self._pii_userid = self._generate_pii_userid()
            return self._pii_userid
    
    def _generate_pii_userid(self) -> str:
        userid = str(np.random.randint(low=self.num_low, high=self.num_high))
        # 0~10 following beta distribution
        prefix_length = int(np.round(np.random.beta(self.beta_a, self.beta_b) * 10))
        prefix = ""
        for i in range(prefix_length):
            prefix += random.choice(string.ascii_letters)
        if prefix:
            prefix += random.choice(".:-_")
        return prefix + userid

In [7]:
class CustomFaker(Faker, UserIdMixIn):
    ...
    
fake = CustomFaker("en_US")

In [8]:
extract_real_words("ID_NUM")

{'014674070485',
 '027693',
 '034626995785',
 '046922558887',
 '047378465',
 '054176622314',
 '06EYD876',
 '132305666219',
 '141774671173',
 '143860010348',
 '159531167997',
 '163133980712',
 '167695383458',
 '172801513686',
 '183169291463',
 '186941941714',
 '188408534931',
 '208798413907',
 '264945858442',
 '276795361801',
 '320622779078',
 '342998677810',
 '347376430553',
 '35615904922',
 '369615882777',
 '409046248321',
 '493950392533',
 '51,00,23,0',
 '522233062166',
 '530670102508',
 '534516353860',
 '557349702179',
 '674915248960',
 '696135165639',
 '723847538279',
 '747051878431',
 '762035863358',
 '779875708882',
 '784372734211',
 '789323889085',
 '792389774673',
 '800306846075',
 '836172426340',
 '843756944804',
 '860632713425',
 '871483046449',
 '875673967537',
 '932353568953',
 '943063077874',
 '943995368223',
 '955487471144',
 '982645662261',
 'DM:705244534902',
 'Ei:556799175487',
 'IV-8322',
 'Iz.:999893751750',
 'Kh:217952887271',
 'Kh:360595695159',
 'Kl.:838901042770'

In [9]:
for _ in range(50):
    print(CustomFaker("en_US").pii_userid)

Gh.822495924264
ZTu.889952675897
827206573138
UIDj:612108100306
mMteP_435620561000
NvH_334401585291
oC.947919481669
QxRDSx:170824388057
Cy.713700154341
jgY.509886285261
yuy_811586291603
n_121560362278
254706470495
b.808853177757
865910089644
hlf.436417500340
vz-667321274751
BLx_772544722534
Grm:275012911529
k.932921693473
Yt.259543347772
gzn:573053885994
N-994716908871
C:173080696644
538273168194
GcOr_290269561191
Yd.191519249687
zAu:520498504792
f-240254946100
oU-198533293294
592234527415
825993558762
q:606993855390
i:659571443785
OS_435004151148
YEVes-425757717044
628106628550
637644337280
f-600201147532
Hl-294437722208
475929908304
HKr.685253776192
Fx-270431197971
871896257556
C-298113318889
863859825829
B.484036763693
pz:998964183123
W.720231716471
NAdM:623719471551


#### Username

In [10]:
class UserNameMixIn:
    username_use_faker_p: float = 0.5
    
    @property
    def pii_username(self) -> str:
        try:
            return self._pii_username
        except AttributeError:
            self._pii_username = self._generate_pii_username()
            return self._pii_username
        
    def _generate_pii_username(self) -> str:
        if random.random() < self.username_use_faker_p:
            return self.user_name()
        else:
            return self.first_name().lower() + self.last_name().lower()

In [11]:
class CustomFaker(Faker, UserNameMixIn):
    ...
    
fake = CustomFaker("en_US")
print(fake.pii_username)
print(fake.pii_username)

jennifergreen
jennifergreen


In [12]:
extract_real_words("USERNAME")

{'castanedagabriel', 'fdixon', 'holmespatrick', 'jacob59', 'meyermichelle'}

In [13]:
for _ in range(10):
    print(CustomFaker("en_US").pii_username)

thomaswilliams
cynthiacook
hhunt
ruth70
aliciamarshall
justin33
melissachapman
rbarajas
williammoody
alyssawashington


#### URL

In [14]:
class UrlMixIn:
    url_use_fake_p: float = 0.5
        
    @property
    def pii_url(self) -> str:
        try:
            return self._pii_url
        except AttributeError:
            self._pii_url = self._generate_pii_url()
            return self._pii_url
        
    def _generate_pii_url(self) -> str:
        if random.random() < self.url_use_fake_p:
            return self.uri()
        if random.random() < 0.5:
            return self._generate_fake_youtube_url()
        return self._generate_fake_sns_url(self)
        
    @staticmethod
    def _generate_fake_youtube_url() -> str:
        # youtube uses random 11 charactors using base64 encoding
        domain = random.choice(["youtube.com/watch?v=", "youtu.be/"])
        base64_chars = [*(string.digits + string.ascii_letters + "_-")]
        random_base64_chars = "".join(random.choices(base64_chars, k=11))
        return f"https://www.{domain}{random_base64_chars}"
    
    @staticmethod
    def _generate_fake_sns_url(fake_person) -> str:
        domain = random.choice(
            ["linkedin.com", "liknkedin.com/in", "facebook.com", "twitter.com"]
        )
        username = fake_person.user_name()
        return f"https://www.{domain}/{username}"

In [15]:
class CustomFaker(Faker, UrlMixIn):
    ...
    
fake = CustomFaker("en_US")
print(fake.pii_url)
print(fake.pii_url)

http://kim.net/categories/categorysearch.htm
http://kim.net/categories/categorysearch.htm


In [16]:
extract_real_words("URL_PERSONAL")

{'http://beck-calhoun.biz/bloghome.jsp',
 'http://camacho.biz/blog/main/appfaq.jsp',
 'http://chang.com/tagspost.asp',
 'http://fisher.com/category/wp-contentregister.htm',
 'http://hensley.info/exploreabout.jsp',
 'http://hodge-ramsey.com/tagmain.html',
 'http://hogan.com/app/categoriesindex.php',
 'http://jacobs-fisher.com/listpost.html',
 'http://jones-mendoza.com/blog/search/searchprivacy.php',
 'http://martin.com/app/tags/searchregister.php',
 'http://mcdonald-pope.com/categorycategory.jsp',
 'http://mcneil.org/list/taghomepage.htm',
 'http://moore.com/explore/tagterms.php',
 'http://osborne.org/main/posts/tagprivacy.asp',
 'http://owens-farley.com/blogprivacy.asp',
 'http://perry-king.biz/main/exploreregister.jsp',
 'http://thomas.biz/list/taghome.php',
 'http://www.barry.com/main/list/postshomepage.html',
 'http://www.bonilla.org/category/tags/wp-contentfaq.htm',
 'http://www.burns-lopez.com/categories/appabout.asp',
 'http://www.chambers.com/wp-content/app/categorieshomepage.js

In [17]:
for _ in range(50):
    print(CustomFaker("en_US").pii_url)

http://www.wolfe.net/explorepost.htm
http://www.miller-aguilar.com/search/appauthor.html
https://www.twitter.com/hebertlisa
https://www.facebook.com/grogers
https://dickerson-henry.com/tagsauthor.html
https://www.liknkedin.com/in/gregorybrad
https://www.liknkedin.com/in/cday
https://www.twitter.com/jeffreymiddleton
https://www.liknkedin.com/in/david23
http://www.pratt.net/tagspost.php
https://www.twitter.com/johnsonchristopher
http://www.johnson.com/searchmain.jsp
https://www.youtu.be/emjIgWxnAFf
https://wilkins-sanchez.com/tagmain.htm
http://www.perez-collier.com/appmain.php
https://www.lee-hill.com/list/listmain.jsp
http://www.moore-neal.com/postspost.html
https://www.youtu.be/WNMCOOblFWY
https://singh.com/explore/wp-content/categorieshomepage.html
https://www.youtube.com/watch?v=HCNmPCoPHnO
https://www.parrish.com/explore/list/exploreabout.jsp
https://www.youtube.com/watch?v=MbY2RdFnyCa
http://www.davis-morris.org/main/blog/wp-contentterms.php
https://jones.com/tag/app/mainhomepage.

In [18]:
fake.street_address()

'62161 Kathleen Meadow Suite 617'

In [19]:
class FixPiiMixIN:
    @property
    def pii_first_name(self):
        try:
            return self._pii_first_name
        except AttributeError:
            self._pii_first_name = self.first_name()
            return self._pii_first_name
    
    @property
    def pii_last_name(self):
        try:
            return self._pii_last_name
        except AttributeError:
            self._pii_last_name = self.last_name()
            return self._pii_last_name
        
    @property
    def pii_street_address(self):
        try:
            return self._pii_street_address
        except AttributeError:
            self._pii_street_address = self.street_address()
            return self._pii_street_address

In [20]:
class Student(Faker, UserIdMixIn, UserNameMixIn, UrlMixIn, FixPiiMixIN):
    def __repr__(self) -> str:
        return f"<Student [{self.pii_first_name} {self.pii_last_name}]>"

In [21]:
def generate_student(n: int = 1) -> list[Student]:
    COUNTRIES = [
        "en_US", "en_US", "en_US", "en_US", "en_US", "en_US",
        "de_DE", "it_IT", "es_ES", "da_DK", "fr_FR", "vi_VN"
    ]
    countries = random.choices(COUNTRIES, k=n)
    return [Student(country) for country in countries]

In [22]:
generate_student(10)

[<Student [Armando Vilar]>,
 <Student [Michelle Täsche]>,
 <Student [Larry Cortez]>,
 <Student [Troy Hayes]>,
 <Student [Danny Henriksen]>,
 <Student [Tammy Reese]>,
 <Student [Helfried Gorlitz]>,
 <Student [Hector Brooks]>,
 <Student [Mary Hayes]>,
 <Student [Candace Moore]>]

In [23]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Generate essays using GPT-4

In [24]:
prompt = Template(
"""Write an essay with following instructions.
Select one of the four design thinking tools presented in the course (listed below) that you are interested in applying to a challenge  of your choice. Your completed reflection should be written in English and be between five and ten paragraphs in length.
Tools: Visualization (Module 1), Storytelling (Module 2), Mind Mapping (Module 3) or Learning Launch (Module 4).  
Elements:
- Challenge: Describe your challenge, including all relevant information.
- Selection: In your own words, briefly describe the tool you selected (e.g., what it is and why you selected it for your  challenge -- including any appropriate video lecture references).
- Application: Describe how you applied the tool you selected to your challenge (e.g., what you did and how the tool was  applied effectively or ineffectively).
- Insight: Describe the insight you gained from applying the tool you selected to your challenge (e.g., how an insight  affected your thinking about the challenge and about design thinking more broadly).
- Approach: Describe what you might do differently next time -- applying the same tool you selected or a different one - - and the reason(s) why.
Reference:
You should include reference(s) in the essay.

Remember to include all the following information about yourself in the essay as naturally as possible.
Name: {{first_name}} {{last_name}}
User ID: {{userid}}
URL: {{url}}
Username: {{username}}
Address: {{street_address}}
"""
)

In [25]:
responses = []
generated = []
async_client = AsyncOpenAI(api_key=UserSecretsClient().get_secret("OPENAI_API_KEY"))
students = generate_student(n=N_GENERATE_ESSAY)

for student in students:
    piis = {
        "first_name": student.pii_first_name, 
        "last_name": student.pii_last_name,
        "userid": student.pii_userid,
        "url": student.pii_url,
        "username": student.pii_username,
        "street_address": student.pii_street_address,
    }
    content = prompt.render(**piis)
    response = async_client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a university student."},
            {"role": "user", "content": content},
        ]
    )
    generated.append(piis)
    responses.append(response)

responses = await asyncio.gather(*responses)

for i, response in enumerate(responses):
    generated[i].update({"full_text": response.choices[0].message.content})

In [26]:
generated_df = pd.DataFrame(generated)
generated_df.to_json("generated_df.json", index=False)